# **Time Series Assignment**

> Subject: Quantitative Investing

> Authors: Hamza Ghoussy, Andreas Koulopoulos, Milan Peter and Shubham Vijay Nanewar

Can we forecast the equity risk premium? In this group assignment we will analyze whether different candidate predictor variables can forecast the equity risk premium. We address this question in three parts: first, we analyze whether the predictor variables can be used to forecast the equity risk premium in-sample. Second, we analyze their predictive performance out-of-sample. Finally, we evaluate whether combining the variables improves the out-of-sample forecasting performance.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm


## **General Setup**

The dataset *predictor_data.csv* contains the (quarterly) Welch and Goyal (2008) predictor variables. In this assignment, we use $N = 14$ out of the $15$ variables considered by Welch and Goyal (2008) and Rapach et al. (2010) as predictor variables:

1. Log dividend-price ratio, D/P (`logDP`): Difference between the log of a 12-month moving sum of dividends paid on the S&P 500 index and the log of stock prices (S&P 500 index).
2. Log dividend yield, D/Y (`logDY`): Difference between the log of a 12-month moving sum of dividends and the log of lagged stock prices.
3. Log earnings-price ratio, E/P (`logEP`): Difference between the log of a 12-month moving sum of earnings on the S&P 500 and the log of stock prices.
4. Log dividend-payout ratio, D/E (`logDE`): Difference between the 12-month moving sums of dividends and earnings.
5. Stock Variance, SVAR (`svar`): Monthly sum of squared daily returns on the S&P 500.
6. Book-to-market ratio, B/M (`b/m`): Ratio of the book value to the market value of the Dow Jones Industrial Average.
7. Net equity expansion, NTIS (`ntis`): Ratio of a 12-month moving sum of net equity issues by NYSE-listed stocks to the total end-of-year market capitalization of NYSE stocks.
8. Treasury bill rate, TBL (`tbl`): Interest rate on a three-month treasury bill on the secondary market.
9. Long-term yield, LTY (`lty`): Long-term yield of government bonds.
10. Long-term returns, LTR (`ltr`): Return on long-term government bonds.
11. Term spread, TMS (`tms`): Difference between the long-term yield and the Treasury bill rate.
12. Default yield spread, DFY (`dfy`): Difference between BAA- and AAA-rated corporate bond yields.
13. Default return spread, DFR (`dfr`): Difference between long-term corporate bond and long-term government bond returns.
14. Inflation, INFL (`lagINFL`): Inflation is calculated from the CPI (all urban consumers). Following Welch and Goyal (2008) and Rapach et al. (2010) we use $x_{i,t−1}$ for inflation since the data are released in the following month.

The empirical framework is based on Welch and Goyal (2008), Rapach et al. (2010), and Rapach et al. (2016) and relies on the predictive regression model for the equity premium given by

$$
r_{t+1} = α_i + β_i x_{i,t} + ε_{t+1} \tag{1}
$$

where $r_{t+1}$ is the return of the stock market index in excess of the risk-free interest rate, $x_{i,t}$ is the predictive variable of interest, and $ε_{t+1}$ denotes the disturbance term. Furthermore, $i$ indexes the forecast variable of interest.

In [9]:
# Load data
dfDATA = pd.read_csv("predictor_data.csv")

# List of predictor variables
lPREDICTORS = ['logDP', 'logDY', 'logEP', 'logDE', 'svar', 'b/m', 'ntis', 'tbl', 'lty', 'ltr', 'tms', 'dfy', 'dfr', 'lagINFL']

# Adjust sign for predictors where a negative relationship with returns is expected
lNEGATIVE = ['ntis', 'tbl', 'lty', 'lagINFL']

for p in lPREDICTORS:
    if p in lNEGATIVE:
            dfDATA[[p]] = -dfDATA[[p]]

# Create lead return variable
dfDATA["r_lead"] = dfDATA["r"].shift(-1)

# Drop rows with missing values
dfDATA = dfDATA.dropna(subset=["r_lead"]).reset_index(drop=True)

## **Section 1**

We perform an in-sample estimation of the predictive regression in equation $(1)$ for each $i = 1, . . . , N$. In our estimation, we take the negative of the predictor variables `ntis`, `tbl`, `lty`, and `lagINFL`, as specified in the question. We, then test the null hypothesis $H_0 : β_i = 0$ against the one-sided alternative $H_A : β_i > 0$. We made sure to report the estimated coefficients, heteroscadesticity and autocorrelation consistent $t$-statistics and the corresponding $p$-values. We also report the adjusted $R^2$ of our regressions.

In [10]:
# model each predictor variable
# save in a dataframe

results = pd.DataFrame(columns=['Predictor', 'alpha', 'beta', 't-stat', 'p-value', 'adj. R2'])

for p in lPREDICTORS:

    y = dfDATA['r_lead']
    X = sm.add_constant(dfDATA[[p]])

    # create the regression model
    # with HAC (Heteroskedasticity and Autocorrelation Consistent)
    model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 4})

    t_stat = model.tvalues[p]
    df_resid = int(model.df_resid)

    # one-sided p-value (H1: beta > 0)
    p_one = 1 - stats.t.cdf(t_stat, df_resid)

    # save results in dataframe
    results.loc[p, 'Predictor'] = p
    results.loc[p, 'alpha'] = model.params['const']
    results.loc[p, 'beta'] = model.params[p]
    results.loc[p, 't-stat'] = t_stat
    results.loc[p, 'p-value'] = p_one
    results.loc[p, 'adj. R2'] = model.rsquared_adj

# display results
print(results.reset_index(drop=True))

   Predictor     alpha      beta    t-stat   p-value   adj. R2
0      logDP  0.073147  0.016918  1.156706  0.124058   0.00339
1      logDY  0.066959  0.015166  1.176557  0.120053  0.002087
2      logEP  0.078286  0.022743  1.553994  0.060507  0.005995
3      logDE  0.014036  -0.00232 -0.093785  0.537335 -0.002563
4       svar  0.014355  0.135355   0.24206  0.404432 -0.002248
5        b/m  -0.01151  0.048654  1.681117  0.046779  0.013268
6       ntis  0.023962  0.525127  1.877998  0.030572  0.013928
7        tbl  0.023189  0.232338   1.49274  0.068165  0.002054
8        lty  0.025689  0.204795  1.250833  0.105881  0.000432
9        ltr  0.013164  0.172413  1.639831  0.050932  0.003737
10       tms   0.00984  0.342349  0.904624  0.183117 -0.000831
11       dfy  0.009008  0.581775  0.345433  0.364979 -0.001145
12       dfr  0.015613 -0.062966 -0.204154  0.580829 -0.002417
13   lagINFL  0.019012  0.466832  0.681585  0.247957  0.000738


## **Section 2**

We perform an out-of-sample estimation of our model, using an expanding window. For this part, we divide the total sample of $T$ observations into an in-sample period consisting of the first $m$ observations, a holdout out-of-sample period consisting of $p$ observations, and an out-of-sample period composed of the last $q$ observations. In general, the equity premium forecast
based on equation $(1)$ is given by

$$
r_{i,t+1} = \hat{α}_{i,t} + \hat{β}_{i,t}x_{i,t} \tag{2}
$$

with $\hat{α}_{i,t}$ and $\hat{β}_{i,t}$ denoting the OLS estimates of $α_i$ and $β_i$, respectively, based on data up to and including period $t$. Thus, using the OLS estimates $\hat{α}_{i,m+p}$ and $\hat{β}_{i,m+p}$ of $α_i$ and $β_i$, respectively, the first out-of-sample forecast is given by

$$
\hat{r}_{i,m+p+1} = \hat{α}_{i,m+p} + \hat{β}_{i,m+p}x_{i,m+p} \tag{3}
$$

In particular, $\{r_{t}\}_{t=2}^{m+p}$ is regressed on a constant, and $\{x_{i,t}\}_{t=1}^{m+p-1}$ following equation $(1)$. The second out-of-sample forecast is generated similarly, by regressing $\{r_{t}\}_{t=2}^{m+p+1}$ on a constant and $\{x_{i,t}\}_{t=1}^{m+p}$ which yields

$$
\hat{r}_{i,m+p+2} = \hat{α}_{i,m+p+1} + \hat{β}_{i,m+p+1}x_{i,m+p+1} \tag{4}
$$

Following this procedure, we obtain $q$ out-of-sample forecasts $\{\hat{r}_{t}\}_{t=m+p+1}^{T}$. We set $m$ to 20 years and $p$ to 10 years in our implementation, as specified in the introduction. In our forecast evaluation, we compare these forecasts with the historical average of the excess returns over the risk-free rate, which serves as a benchmark. In particular, the benchmark return in period $t + 1$ is given by

$$
\bar{r}_{t+1} = \frac{1}{t} \sum_{j=1}^{t} r_{j} \tag{5}
$$

To evaluate the out-of-sample performance of the different forecasts, we use the out-of-sample $R^2$, $R^2_{OS}$ (Campbell and Thompson, 2008). The out-of-sample $R^2$ compares the forecast based on the predictive regression model in equation $(1)$ $\hat{r}_{t+1}$, with the benchmark $\bar{r}_{t+1}$. In particular, it measures the proportional reduction in MSPE for $\hat{r}_{t+1}$ relative to $\bar{r}_{t+1}$:

$$
R^2_{OS} = 1 −
\frac{\sum_{k=p+1}^{p+q} (r_{m+k} − \hat{r}_{m+k})^2}{\sum_{k=p+1}^{p+q} (r_{m+k} − \bar{r}_{m+k})^2} \tag{6}
$$

where $q$ are the out-of-sample observations (see above). A positive out-of-sample $R^2$ indicates that the forecast $\hat{r}_{t+1}$ outperforms the historical average forecast in terms of MSPE. We report the out-of-sample $R^2$.

In [ ]:
# Parameters for out-of-sample forecasting
iM = 20 * 4
iP = 10 * 4
iT = len(dfDATA)

# Out-of-sample R^2_OS for each predictor
r2_oos = {}
pred_detail = {}

for p in lPREDICTORS:
    preds, benches, reals = [], [], []

    # Forecast origins t = iM+iP, ..., iT-1 (because the last row is NaN after shifting)
    for t in range(iM + iP, iT - 1):

        # Training window: rows [0, t)
        dfTRAIN = dfDATA.iloc[:t]

        y_train = dfTRAIN["r_lead"]
        X_train = sm.add_constant(dfTRAIN[[p]], has_constant='add')

        model = sm.OLS(y_train, X_train).fit(cov_type='HAC', cov_kwds={'maxlags': 4})

        # Forecast r_{t+1} using x_t (row t)
        X_next = sm.add_constant(dfDATA[[p]].iloc[t:t+1], has_constant='add')
        y_pred = float(model.predict(X_next).iloc[0])

        # Realized r_{t+1} is r_lead at row t
        y_real = float(dfDATA["r_lead"].iloc[t])

        # Benchmark: historical mean of realized returns up to t
        bench_pred = float(dfDATA["r"].iloc[:t+1].mean())

        # Store results 
        preds.append(y_pred)
        benches.append(bench_pred)
        reals.append(y_real)

    # Convert to numpy arrays
    preds = np.array(preds)
    benches = np.array(benches)
    reals = np.array(reals)

    # Compute R^2_OS by Campbell & Thompson (2008)
    ssr_model = np.sum((reals - preds) ** 2)
    ssr_bench = np.sum((reals - benches) ** 2)
    r2_oos[p] = 1 - ssr_model / ssr_bench

    # Store results for this predictor
    pred_detail[p] = {"preds": preds, "bench": benches, "reals": reals}

# Report: which variables beat the benchmark?
r2_series = pd.Series(r2_oos).sort_values(ascending=False)
winners = r2_series[r2_series > 0]

print("Out-of-sample R^2_OS by predictor (expanding window):")
print(r2_series)

if len(winners) == 0:
    print("\nNo predictor beats the historical-mean benchmark (R^2_OS <= 0).")
else:
    print("\nPredictors that outperform the benchmark (R^2_OS > 0):")
    print(winners)


Out-of-sample R^2_OS by predictor (expanding window):
lagINFL    0.005288
ltr        0.004127
dfy        0.001243
tbl       -0.001924
svar      -0.003549
tms       -0.003748
logDY     -0.005897
logDP     -0.015612
lty       -0.025584
ntis      -0.033902
logDE     -0.042463
dfr       -0.045812
logEP     -0.063077
b/m       -0.071210
dtype: float64

Predictors that outperform the benchmark (R^2_OS > 0):
lagINFL    0.005288
ltr        0.004127
dfy        0.001243
dtype: float64


## **Section 3**

In this section, we consider the out-of-sample performance of a so-called ”kitchensink” regression in which we include all predictor variables.

In [ ]:
# Kitchensink out-of-sample (expanding window) forecast and R^2_OS

preds_k, benches_k, reals_k = [], [], []

# Expanding-window kitchensink (fit all predictors each period)
for t in range(iM + iP, iT - 1):
    # training sample uses rows [0, t) -> info up to and including time t
    dfTRAIN = dfDATA.iloc[:t]

    y_train = dfTRAIN["r_lead"]
    X_train = sm.add_constant(dfTRAIN[lPREDICTORS], has_constant='add')
    
    model_ks = sm.OLS(y_train, X_train).fit(cov_type='HAC', cov_kwds={'maxlags': 4})

    # one-step-ahead forecast of r_{t+1} using x_t (row t)
    X_next = sm.add_constant(dfDATA[lPREDICTORS].iloc[t:t+1], has_constant='add')
    y_pred = float(model_ks.predict(X_next).iloc[0])

    # realized r_{t+1}
    y_real = float(dfDATA["r_lead"].iloc[t])

    # recursive benchmark mean up to t: \bar r_{t+1} = (1/t) sum_{j=1}^t r_j
    bench_pred = float(dfDATA["r"].iloc[:t+1].mean())

    preds_k.append(y_pred)
    benches_k.append(bench_pred)
    reals_k.append(y_real)

preds_k = np.array(preds_k)
benches_k = np.array(benches_k)
reals_k = np.array(reals_k)

# 3) Out-of-sample R^2_OS (Campbell & Thompson)
ssr_model_k = np.sum((reals_k - preds_k) ** 2)
ssr_bench_k = np.sum((reals_k - benches_k) ** 2)
r2_kitchensink = 1 - ssr_model_k / ssr_bench_k

print("Kitchensink (expanding window) R^2_OS:", r2_kitchensink)


Kitchensink (expanding window) R^2_OS: -0.2661835047017449


## **Section 4**

In the fourth section, we stay in the out-of-sample setting. Individual forecasts may be improved by combining all individual forecasts into a combined forecast. We combined the individual forecasts for period $t + 1$ computed based on equation $(1)$ as follows

$$
\hat{r}_{c,t+1} = \sum_{i=1}^{N} \omega_{i,t} \hat{r}_{i,t+1}, \tag{7}
$$

where $\omega_{i,t}$ denotes the ex-ante combination weight for the forecast based on variable $i$ at time $t$. Theoretically, $\omega_{i,t}$ can vary over $t$, however, they do not need to.

First, we consider averaging methods of the individual forecast. For the mean combination forecast, we set the combination weights in equation $(7)$ $\omega_{i,t} = 1/N$ for each $i = 1, . . . , N$. Next, for the median combination forecast, select the median of $\{\hat{r}_{i,t+1}\}_{i=1}^{N}$. That is, set $\omega_{j,t} = 1$ for the forecast variable $j \in \{1, . . . , N \}$ associated with the individual forecast being the median forecast $\hat{r}_{j,t+1}$ of $\{\hat{r}_{i,t+1}\}_{i=1}^{N}$. Otherwise, we set $\omega_{j,t} = 0$.

Using the discount mean square prediction error (DMSPE) method allows the combination weights to depend on the forecast performance of the individual variables over the holdout out-of-sample period (Stock and Watson, 2006). In particular, the combining weights are calculated as

$$
\omega_{i,t} = \Phi^{-1}_{i,t} / \sum_{j=1}^{N} \Phi^{-1}_{j,t} \text{, where    }
\Phi_{i,t} = \sum_{s=m}^{t-1} \theta^{t-1-s} (r_{s+1} - \hat{r}_{i,s+1})^2 \tag{8}
$$

with $\theta > 0$ being a discount factor, $m + 1$ being the first period in the holdout out-of-sample period, and the individual forecast $\hat{r}_{i,s+1}$ is based on data up to and including period $s$.

Therefore, the DMSPE method attaches greater weight to forecast variables whose forecasts yielded lower MSPEs over the holdout out-of-sample period. Furthermore, $\theta$ controls the degree of discounting. Report results for $\theta = 0.9$ and $\theta = 1$.

In [ ]:
# Forecast combinations

# Fit individual predictor models once on in-sample + holdout
dfTRAIN = dfDATA.iloc[:iM + iP].copy()
dfOOS = dfDATA.iloc[iM + iP:iT].copy()

y_train = dfTRAIN['r_lead'].to_numpy()
y_oos = dfOOS['r_lead'].to_numpy()
T_oos = len(y_oos)

individual_preds = []  # will store forecasts per predictor

for pred in lPREDICTORS:
    X_train = sm.add_constant(dfTRAIN[[pred]], has_constant='add')
    model_i = sm.OLS(y_train, X_train).fit(cov_type='HAC', cov_kwds={'maxlags': 4})

    X_oos = sm.add_constant(dfOOS[[pred]], has_constant='add')
    y_pred = model_i.predict(X_oos).to_numpy()
    individual_preds.append(y_pred)

# Shape: (N predictors, T_oos)
individual_preds = np.vstack(individual_preds)

# Combine forecasts
# Mean forecast
mean_comb = np.mean(individual_preds, axis=0)

# Median forecast
median_comb = np.median(individual_preds, axis=0)

# DMSPE weights
def dmspe_weights(individual_preds, y_oos, theta=1.0):
    N, T = individual_preds.shape
    weights = np.zeros((N, T))
    comb_forecast = np.zeros(T)

    # start after first OOS obs
    for t in range(T):
        # compute discounted MSPE up to t
        Phi = []
        for i in range(N):
            errors_sq = (y_oos[:t] - individual_preds[i, :t])**2 if t > 0 else np.array([1e6])
            discounts = theta**np.arange(t-1, -1, -1) if t > 0 else np.array([1.0])
            Phi_i = np.sum(discounts * errors_sq)
            Phi.append(Phi_i)

        Phi = np.array(Phi)
        invPhi = 1 / Phi
        omega = invPhi / invPhi.sum()

        weights[:, t] = omega
        comb_forecast[t] = (omega @ individual_preds[:, t])

    return comb_forecast, weights

# DMSPE forecasts
dmspe_comb_1, w1 = dmspe_weights(individual_preds, y_oos, theta=1.0)
dmspe_comb_09, w09 = dmspe_weights(individual_preds, y_oos, theta=0.9)

# Evaluate R^2_OS for each combination
def r2_os(y_true, y_pred):
    ssr_model = np.sum((y_true - y_pred)**2)
    ssr_bench = np.sum((y_true - y_true.mean())**2)  # static mean benchmark
    return 1 - ssr_model/ssr_bench

print("Mean comb R^2_OS:", r2_os(y_oos, mean_comb))
print("Median comb R^2_OS:", r2_os(y_oos, median_comb))
print("DMSPE θ=1 R^2_OS:", r2_os(y_oos, dmspe_comb_1))
print("DMSPE θ=0.9 R^2_OS:", r2_os(y_oos, dmspe_comb_09))


Mean comb R^2_OS: -0.005495827047952817
Median comb R^2_OS: -0.0020379546498741075
DMSPE θ=1 R^2_OS: -0.002467204620882235
DMSPE θ=0.9 R^2_OS: 0.0013940223680855013


## **References**

Campbell, J. Y. and Thompson, S. B. (2008). Predicting excess stock returns out of sample: Can anything beat the historical average? The Review of Financial Studies, 21(4):1509–1531.

Rapach, D. E., Ringgenberg, M. C., and Zhou, G. (2016). Short interest and aggregate stock returns. Journal of Financial Economics, 121(1):46–65.

Rapach, D. E., Strauss, J. K., and Zhou, G. (2010). Out-of-sample equity premium prediction: Combination forecasts and links to the real economy. The Review of Financial Studies, 23(2):821–862.

Stock, J. H. and Watson, M. W. (2006). Forecasting with many predictors. Handbook of Economic Forecasting, 1:515–554.

Welch, I. and Goyal, A. (2008). A comprehensive look at the empirical performance of equity premium prediction. The Review of Financial Studies, 21(4):1455–1508.